In [ ]:
#test할 전체 이미지에서 patch 뜯어서 가져오고, patch별로 어디서 뜯어왔는지 보여주는 ratio index array 반환
def val_get_patches(extractor,i,w_patch,h_patch,ratio_threshold,mask_size=1,org_level=2):
    img=extractor.obj_slide_list[i]
    
    LEVEL_EXTRACTION=2
    if float(img.properties['aperio.MPP']) > 0.3:
        scale_factor = 1
    else:
        scale_factor = 2
    
    #print('scale_factor:{}'.format(scale_factor))
    w_4,h_4=img.level_dimensions[LEVEL_EXTRACTION]
    w_0,h_0=img.level_dimensions[0]
    
    img_np=np.array(img.read_region((0,0),LEVEL_EXTRACTION,(w_4,h_4)))[:,:,0:3]
    #print('img_np size:{}'.format(img_np.shape))
    img_np=cv2.resize(img_np,(int(w_4/scale_factor),int(h_4/scale_factor)))
    
    #print('img_np size after resize:{}'.format(img_np.shape))
    
    img_gray=cv2.cvtColor(img_np,cv2.COLOR_RGB2GRAY)

    pad_h=h_patch-img_gray.shape[0]%h_patch
    pad_w=w_patch-img_gray.shape[1]%w_patch


    img_gray=np.pad(img_gray,((0,pad_h),(0,pad_w)),'constant', constant_values=(255,255))
    #print('size after padding:{}'.format(img_gray.shape))
    _,img_gray=cv2.threshold(img_gray,235,255,cv2.THRESH_BINARY_INV)
    
    n_patch_h=img_gray.shape[0]//h_patch
    n_patch_w=img_gray.shape[1]//w_patch


    img_gray=img_gray.reshape(
        n_patch_h,
        h_patch,
        n_patch_w,
        w_patch 
        )

    img_gray=img_gray.transpose((0,2,1,3))
    
    
    
    ratio=(img_gray.reshape(img_gray.shape[0],img_gray.shape[1],-1)==255).sum(-1)/(h_patch*w_patch)
    ratio_index=np.where(ratio>=ratio_threshold,True,False)
    

    org_scale=4**(2-org_level)
    org_img_list=[]

    #fig,axes=plt.subplots (n_patch_h,n_patch_w)

    for i in range(n_patch_h):
        for j in range(n_patch_w):
            if ratio_index[i][j]==True:
                #print(i,j)
                #print('ratio_index:{}'.format(ratio_index[i][j]))
            
                org_img=np.array(img.read_region((j*w_patch*16*scale_factor,i*h_patch*16*scale_factor),org_level,(w_patch*scale_factor*org_scale,h_patch*scale_factor*org_scale)))[:,:,0:3]
                org_img=cv2.resize(org_img,(w_patch*org_scale,h_patch*org_scale)) #org_img의 사이즈를 mpp (scale_factor)로 보정
                #mask가 layer2 크기로 설정되어 있어 만약 level이 2가 아니면 mask보다 org_img 크기가 더 커짐.
                
                #axes[i][j].imshow(org_img) ##org_img 시각화
                #axes[i][j].imshow(layer3_patch,'gray') ##layer3시각화
                org_img_list.append(org_img)
            #else:
                #axes[i,j].imshow(np.zeros((w_patch,h_patch)))  
    
    if len(org_img_list)>0:
        org_img_np=np.stack(org_img_list,axis=0)
        
        return org_img_np,ratio_index,1
    else:
        return 0,0,0,0

In [ ]:
import argparse
import numpy as np
from PIL import Image
from scipy.linalg import eigh


def normalizeStaining(img, saveFile=None, Io=240, alpha=1, beta=0.15):
    ''' Normalize staining appearence of H&E stained images
    
    Example use:
        see test.py
        
    Input:
        I: RGB input image
        Io: (optional) transmitted light intensity
        
    Output:
        Inorm: normalized image
        H: hematoxylin image
        E: eosin image
    
    Reference: 
        A method for normalizing histology slides for quantitative analysis. M.
        Macenko et al., ISBI 2009
    '''
             
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
        
    maxCRef = np.array([1.9705, 1.0308])
    
    # define height and width of image
    h, w, c = img.shape
    
    # reshape image
    img = img.reshape((-1,3))

    # calculate optical density
    OD = -np.log((img.astype(np.float)+1)/Io)
    
    # remove transparent pixels
    ODhat = OD[~np.any(OD<beta, axis=1)]
        
    # compute eigenvectors
    cov=np.cov(ODhat.T)
    if np.isnan(cov).sum()!=0:
        return img.reshape(256,256,3)
    eigvals, eigvecs = eigh(cov)
    
    #eigvecs *= -1
    
    #project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3])
    
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    # recreate the image using reference mixing matrix
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  

    return Inorm

In [ ]:
#input으로 extractor, load한 model, predict할 이미지의 index넣으면, predictiond이랑 해당 이미지에서 ratio index 반환하는 함수
def prediction (extractor,model,index,threshold=0.1,w_patch=256,h_patch=256):
    test_img_np,ratio_index,info=val_get_patches(extractor,index,h_patch,w_patch,threshold)
    test_img_norm=np.zeros(test_img_np.shape)
    for i in range(len(test_img_norm)):
        test_img_norm[i]=normalizeStaining(test_img_norm[i])
        test_img_norm[i]=keras.utils.normalize(test_img_np[i]/255,axis=-1)
    prediction = model.predict(test_img_norm)
    
    return prediction,ratio_index

In [ ]:
def unpatch(pred,ratio_index):
    n_patch_h=int(ratio_index.shape[0])
    n_patch_w=int(ratio_index.shape[1])
    
    w_patch=int(pred.shape[2])
    h_patch=int(pred.shape[1])
    
    layer3_np=np.zeros((n_patch_h*h_patch,n_patch_w*w_patch,1))
    layer5_np=np.zeros((n_patch_h*h_patch,n_patch_w*w_patch,1))
    
    count=0
    for i in range(n_patch_h):
        for j in range(n_patch_w):
            if ratio_index[i][j]==True:
                layer3_np[i*h_patch:(i+1)*h_patch,j*w_patch:(j+1)*w_patch,0]=pred[count,:,:,0]
                layer5_np[i*h_patch:(i+1)*h_patch,j*w_patch:(j+1)*w_patch,0]=pred[count,:,:,1]
                count+=1
            else:
                layer3_np[i*h_patch:(i+1)*h_patch,j*w_patch:(j+1)*w_patch,0]=np.zeros((h_patch,w_patch))
                layer5_np[i*h_patch:(i+1)*h_patch,j*w_patch:(j+1)*w_patch,0]=np.zeros((h_patch,w_patch))
    
    
    return layer3_np,layer5_np

In [ ]:
def thresholding(mask,threshold):
    output=mask.copy()
    output[output>=threshold]=255
    output[output<threshold]=0
    return output

In [ ]:
def get_result (extractor,model,threshold=0.5):
    df_np=np.zeros((len(extractor.base_path_list), 2))
    for i in range(len(extractor)):
        pred_mask,ratio=prediction(extractor,model,index=i)
        layer3,layer5=unpatch(pred_mask,ratio)
        layer3_th=thresholding(layer3,0.5)
        layer5_th=thresholding(layer5,0.5)
    
        PD_muscle_volume=Task1_handling.get_quanti_image(layer3_th,256,0.5)
        PD_ganglion_volume=Task1_handling.get_quanti_image(layer5_th,256,0.5)
    
        df_np[i][0]=PD_muscle_volume
        df_np[i][1]=PD_ganglion_volume
    
    df=pd.DataFrame(np.zeros((len(extractor),3)),columns=['case','PD_muscle_volume','PD_ganglion_volume'])
    case_list=[extractor.base_path_list[i].split('/')[3] for i in range(len(extractor))]
        
    df['case']=case_list
    df[['PD_muscle_volume','PD_ganglion_volume']]=df_np
        
    return df

In [ ]:
model_file='att_unet_bce+tversky_channel_ResNet101_normalized.hdf5'
model =load_model(model_file, compile=False)

In [ ]:
#extractor 생성하기
base_path='/mnt/dataset/*.svs'
extractor=Task1_handling.mask_extractor(base_path)

#TEST
#base_path='/mnt/PATH/dataset'
#extractor=Task1_handling.mask_extractor(base_path)

In [ ]:
index=0
pred_mask,ratio=prediction(extractor,
           model,
          index=index
            )

In [ ]:
layer3,layer5=unpatch(pred_mask,ratio)
layer3_th=thresholding(layer3,0.5)
layer5_th=thresholding(layer5,0.2)
layer_th=np.concatenate((layer3_th,layer5_th),axis=-1)


print('layer3 volume: {}'.format(Task1_handling.get_quanti_image(layer3_th,256,0.5)))
print('layer5 volume: {}'.format(Task1_handling.get_quanti_image(layer5_th,256,0.5)))


layer3_gt,layer5_gt=extractor.mask_extraction(index)

gt_resize=np.zeros((layer3_th.shape[0],layer3_th.shape[1],2))

gt_resize[0:layer3_gt.shape[0],0:layer3_gt.shape[1],0]=layer3_gt
gt_resize[0:layer3_gt.shape[0],0:layer3_gt.shape[1],1]=layer5_gt



In [ ]:
cols=2
rows=4

fig,axes=plt.subplots(cols,rows)


for i in range(cols):
    axes[i][0].imshow(gt_resize[:,:,0],'gray')
    axes[i][1].imshow(layer3_th,'gray')
    axes[i][2].imshow(gt_resize[:,:,1],'gray')
    axes[i][3].imshow(layer5_th,'gray')
    
plt.show()
    
plt.show()

In [ ]:
def IoU_by_labels(gt,pred):
    n_classes=gt.shape[-1]
    by_labels=np.zeros(n_classes)
    

    for i in range(n_classes):
        IoU = MeanIoU(num_classes=n_classes)
        IoU.update_state(gt[:,:,i]/255, pred[:,:,i]/255)
        IoU = IoU.result().numpy()
        by_labels[i]=IoU
    return by_labels

In [ ]:
def vol_ratio(gt,pred):
    
    n_classes=gt.shape[-1]
    by_labels=np.zeros(n_classes)
    
    for i in range(n_classes):
        gt_quanti=Task1_handling.get_quanti_image(gt[:,:,i],256,0.5)
        th_quanti=Task1_handling.get_quanti_image(pred[:,:,i],256,0.5)
        #print(gt_quanti,th_quanti)
        by_labels[i]=th_quanti/gt_quanti
        
    return by_labels

In [ ]:
def metrics (extractor, model, index, th_3,th_5):
    pred_mask,ratio=prediction(extractor,model,index)
    layer3,layer5=unpatch(pred_mask,ratio)
    
    layer3_th=thresholding(layer3,th_3)
    layer5_th=thresholding(layer5,th_5)

    layer_th=np.concatenate((layer3_th,layer5_th),axis=-1)
    layer3_gt,layer5_gt=extractor.mask_extraction(index)
    gt_resize=np.zeros((layer3_th.shape[0],layer3_th.shape[1],2))
    
    gt_resize[0:layer3_gt.shape[0],0:layer3_gt.shape[1],0]=layer3_gt
    gt_resize[0:layer3_gt.shape[0],0:layer3_gt.shape[1],1]=layer5_gt
    
    result_IoU=IoU_by_labels(gt_resize,layer_th)
    result_ratio=vol_ratio(gt_resize,layer_th)
    
    return result_IoU , result_ratio
    

In [ ]:
iou_list=[]
ratio_list=[]
for i in range(70,75):
    metric_iou, metric_ratio=metrics(extractor,model,i,0.5,0.2)
    print('image {}'.format(i))
    print('IoU of layer3: {}'.format(metric_iou[0]))
    print('IoU of layer5: {}'.format(metric_iou[1]))
    
    print('ratio of layer3: {}'.format(metric_ratio[0]))
    print('ratio of layer5: {}'.format(metric_ratio[1]))
    
    print('\n')
    iou_list.append(metric_iou)
    ratio_list.append(metric_ratio)
    
result_iou=np.stack(iou_list)
result_ratio=np.stack(ratio_list)

In [ ]:
total_iou=result_iou.mean(0)
print('total iou')
print('IoU of layer3: {}'.format(total_iou[0]))
print('IoU of layer5: {}'.format(total_iou[1]))

In [ ]:
total_ratio=result_ratio.mean(0)
print('total ratio')
print('ratio of layer3: {}'.format(total_ratio[0]))
print('ratio of layer5: {}'.format(total_ratio[1]))